## 2d. Evidence - Resource Utilization QAS Measurements

Now we collect stored, CPU and memory usage data when predicting with the model, for the Performance scenario.

The cell below must contain JSON data about this evidence that will be used to automatically populate the sample test catalog.

In [ ]:
{
    "tags": ["Computer Vision", "Image"],
    "quality_attribute": "Resource Utilization",
    "description": "Evaluating model anility to run adequately on deployment platform",
    "inputs": "Model",
    "output": "Memory usuage, CPU usuage and disk space",
}

### Initialize MLTE Context

MLTE contains a global context that manages the currently active _session_. Initializing the context tells MLTE how to store all of the artifacts that it produces. This import will also set up global constants related to folders and model to use.

In [ ]:
# Sets up context for the model being used, sets up constants related to folders and model data to be used.
from session import *

### Set up scenario test case

In [ ]:
from mlte.negotiation.artifact import NegotiationCard

card = NegotiationCard.load()
qa = 3
print(card.quality_scenarios[qa])

**A Specific test case generated from the scenario:**

**Data and Data Source:**	The original test dataset can be used (i.e., original test data set aside for testing during data split into training, validation, and test).

**Measurement and Condition:**	    
    1- Executing the model on the loaned platform will not exceed maximum CPU usage of 30% to ensure reasonable response time. CPU usage will be measure using ps.
    2- Memory usage at inference time will not exceed available memory of 512 MB. This will be measured using pmap.
    3- Disk usage will not exceed allocated disk space of 150 MB bytes. This will be measured by adding the size of each file in the path for the model code.

**Context:**	Normal Operation

### Measurements

Prepare and execute measurements.

In [ ]:
from mlte.measurement.storage import LocalObjectSize
from mlte.evidence.types.real import Real
from mlte.measurement.units import Units

store_measurement = LocalObjectSize("model size")
size: Real = store_measurement.evaluate(MODELS_DIR, unit=Units.byte)
print(size)
size.save(force=True)

In [ ]:
from mlte.measurement.process_measurement import ProcessMeasurement
from mlte.measurement.cpu import LocalProcessCPUUtilization, CPUStatistics

cpu_measurement = LocalProcessCPUUtilization("predicting cpu")
cpu_stats: CPUStatistics = cpu_measurement.evaluate(MODEL_COMMAND)
print(cpu_stats)
cpu_stats.save(force=True)

In [ ]:
from mlte.measurement.memory import (
    LocalProcessMemoryUtilization,
    MemoryStatistics,
)

mem_measurement = LocalProcessMemoryUtilization("predicting memory")
mem_stats: MemoryStatistics = mem_measurement.evaluate(MODEL_COMMAND)
print(mem_stats)
mem_stats.save(force=True)

We can also avoid starting the training process twice by using the asynch methods for both measurements. We start the training process once and pass the id to both measurements.

In [ ]:
from mlte.measurement.cpu import LocalProcessCPUUtilization
from mlte.measurement.memory import LocalProcessMemoryUtilization
from mlte.measurement.process_measurement_group import ProcessMeasurementGroup

# Create measurement group
measurements = ProcessMeasurementGroup()

# Add measurements to group.
measurements.add(LocalProcessCPUUtilization("predicting cpu"))
measurements.add(LocalProcessMemoryUtilization("predicting memory"))

# Evaluate the measurements.
evidences = measurements.evaluate(command=MODEL_COMMAND)

# Get results.
cpu_stats = evidences["predicting cpu"]
mem_stats = evidences["predicting memory"]

# Inspect values
print(cpu_stats)
print(mem_stats)

# Save to artifact store
cpu_stats.save(force=True)
mem_stats.save(force=True)